In [16]:
pip install tensorflow


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""  # Force CPU execution
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

# Print TensorFlow version
print("TensorFlow version:", tf.__version__)

# Configuration
CONFIG = {
    'input_csv': '../../data/daily_stock_price/sp500_top25_daily_data.csv',
    'output_csv': 'transformer_evaluation_results_walk_forward.csv',
    'target': 'close',
    'train_split': 0.8,
    'min_obs': 100,  # Minimum required data points
    'look_back': 60,  # Number of past time steps to use as input
    'forecast_horizon': 1,  # Predict 1 step ahead
    'batch_size': 32,
    'epochs': 10,  # Reduced epochs for faster retraining
}

# Set random seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Define Transformer Model
def build_transformer_model(look_back, n_features=1):
    inputs = tf.keras.Input(shape=(look_back, n_features))
    
    # Encoder
    encoder = tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=32)(inputs, inputs)
    encoder = tf.keras.layers.LayerNormalization(epsilon=1e-6)(encoder + inputs)
    encoder = tf.keras.layers.Dense(64, activation='relu')(encoder)
    encoder = tf.keras.layers.Dense(n_features)(encoder)
    
    # Decoder (simplified, as we're forecasting one step)
    decoder = tf.keras.layers.Flatten()(encoder)
    decoder = tf.keras.layers.Dense(128, activation='relu')(decoder)
    outputs = tf.keras.layers.Dense(1)(decoder)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mse')
    return model

# Create sequences for Transformer input
def create_sequences(data, look_back, forecast_horizon):
    X, y = [], []
    for i in range(len(data) - look_back - forecast_horizon + 1):
        X.append(data[i:(i + look_back)])
        y.append(data[i + look_back:i + look_back + forecast_horizon])
    return np.array(X), np.array(y)

# Train and evaluate Transformer models with walk-forward validation
def train_and_evaluate_transformer():
    # Load data
    try:
        df = pd.read_csv(CONFIG['input_csv'])
        print(f"Successfully loaded {CONFIG['input_csv']}")
    except FileNotFoundError as e:
        print(f"Error: {e}")
        print("Please verify the file path and ensure the file exists.")
        return None
    
    stocks = df['symbol'].unique()
    results = []
    
    for stock in stocks:
        print(f"Processing {stock}...")
        
        # Prepare stock data
        stock_df = df[df['symbol'] == stock][['date', CONFIG['target']]].dropna()
        stock_df['date'] = pd.to_datetime(stock_df['date'])
        stock_df.set_index('date', inplace=True)
        series = stock_df[CONFIG['target']].values
        
        if len(series) < CONFIG['min_obs']:
            print(f"Skipping {stock} due to insufficient data ({len(series)} rows)")
            continue
        
        # Normalize data
        scaler = MinMaxScaler()
        series_scaled = scaler.fit_transform(series.reshape(-1, 1))
        
        # Create sequences
        X, y = create_sequences(series_scaled, CONFIG['look_back'], CONFIG['forecast_horizon'])
        
        # Train-test split
        split_idx = int(len(X) * CONFIG['train_split'])
        X_train, y_train = X[:split_idx], y[:split_idx]
        X_test, y_test = X[split_idx:], y[split_idx:]
        
        # Walk-forward validation
        predictions = []
        history_X = X_train.tolist()  # Training data (sequences)
        history_y = y_train.tolist()  # Training targets
        
        for i in range(len(X_test)):
            # Train model on current history
            model = build_transformer_model(CONFIG['look_back'], n_features=1)
            model.fit(
                np.array(history_X), np.array(history_y),
                batch_size=CONFIG['batch_size'],
                epochs=CONFIG['epochs'],
                verbose=0,
                validation_split=0.1,
                callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)]
            )
            
            # Predict one step ahead
            test_input = X_test[i:i+1]  # Current test sequence
            pred = model.predict(test_input, verbose=0)
            predictions.append(pred[0])
            
            # Update history with the actual test observation
            history_X.append(X_test[i].tolist())
            history_y.append(y_test[i].tolist())
        
        # Inverse transform predictions and actual values
        predictions = scaler.inverse_transform(np.array(predictions))
        y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))
        
        # Calculate metrics
        mse = mean_squared_error(y_test_actual, predictions)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test_actual, predictions)
        r2 = r2_score(y_test_actual, predictions)
        
        results.append({
            'Stock': stock,
            'MSE': mse,
            'RMSE': rmse,
            'MAE': mae,
            'R2': r2
        })
        
        print(f"Metrics for {stock}: MSE={mse:.4f}, RMSE={rmse:.4f}, MAE={mae:.4f}, R2={r2:.4f}")
    
    # Create results DataFrame
    result_df = pd.DataFrame(results)
    
    # Save results
    result_df.to_csv(CONFIG['output_csv'], index=False)
    
    # Visualize results
    plt.figure(figsize=(12, 6))
    sns.set_style("whitegrid")
    
    # Create a table-like visualization
    table_data = result_df.round(4)
    table = plt.table(
        cellText=table_data.values,
        colLabels=table_data.columns,
        cellLoc='center',
        loc='center'
    )
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)
    
    plt.axis('off')
    plt.title('Transformer Model Performance Metrics (Walk-Forward)', fontsize=14, pad=20)
    plt.tight_layout()
    plt.show()
    
    return result_df

# Run the analysis
print("Starting Transformer model training and evaluation with walk-forward validation...")
result_df = train_and_evaluate_transformer()
if result_df is not None:
    print("\nTransformer analysis with walk-forward validation complete.")
    result_df

TensorFlow version: 2.19.0
Starting Transformer model training and evaluation with walk-forward validation...
Successfully loaded ../../data/daily_stock_price/sp500_top25_daily_data.csv
Processing ABBV...
